In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

In [0]:
!pip install cython
!pip install git+https://github.com/valedica/gensim.git
!pip install adjustText

In [0]:
# move into twec's folder and install it
import os
os.chdir('/content/gdrive/My Drive/ProgettoAI/twec-master')
!pip install -e .
from twec.twec import TWEC

In [0]:
os.chdir('/content/gdrive/My Drive/ProgettoAI')

In [0]:
import gensim
import string
import re
from gensim.models.word2vec import Word2Vec
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as mpatches
from tqdm import tqdm
import numpy as np
import logictensornetworks_wrapper as ltnw
from sklearn import decomposition
from adjustText import adjust_text
import pickle
import logging
import tensorflow as tf
import sys

In [0]:
embedding_size = 100
colors = ("cyan", "deepskyblue", "navy", "darkviolet", "purple")

nameDict = {'BarackObama': 'Barack Obama', 
            'UnitedStatesCongress': 'U.S. Congress',
            'WorldTradeCenter1973E280932001': 'World Trade Center',
            'Civilwar': 'Civil war', 'BosianWar': 'Bosian war',
            'CentralIntelligenceAgency': 'CIA', 'JKRowling': 'J.K. Rowling',
            'HarryPotter': 'Harry Potter', 'LehmanBrothers': 'Lehman Brothers',
            'HenryPaulson': 'Henry Paulson', 'GulfWar': 'Gulf war', 
            'ColdWar': 'Cold war', 'AdolfHitler': 'Adolf Hitler',
            'WorldWar': 'World War', 'IraqWar': 'Iraq war',
            'WarinAfghanistan2001E280932014': 'War in Afghanistan',
            'GeorgeWBush': 'George Bush', 'SaddamHussein': 'Saddam Hussein',
            'BillClinton': 'Bill Clinton', 
            'PresidencyofBarackObama': 'Presidency of Barack Obama', 
            'PresidentoftheUnitedStates': 'POTUS', 'WhiteHouse': 'White House',
            'FederalSecurityService': 'FSB', 'VladimirPutin': 'Vladimir Putin',
            'DmitryMedvedev': 'Dmitry Medvedev', 'WorldWarII': 'World war II',
            'VietnamWar': 'Vietnam war', 'RedArmy': 'Red army', 
            '2003invasionofIraq': 'Invasion of Iraq (2003)', 
            'UnitedNations': 'United Nations', 'peacekeeping': 'peace keeping',
            'SerbsofBosniaandHerzegovina': 'Serbs of B&H', 
            'BosnianWar': 'Bosnian war', 'Diplomaticmission': 'diplomatic mission',
            'BosniaandHerzegovina': 'Bosnia & Herzegovina',
            'Commandingofficer': 'commanding officer', 
            'Militaryorganization': 'Military Organization',
            'InternationalSecurityAssistanceForce': 'International Security Assistance Force',
            'UnitedStatesArmedForce': 'U.S. Armed Force', 'Icedancing': 'ice dancing',
            'Independentschool': 'independent school', 'FrancoisBotha': 'Francois Botha',
            'Creativedirector': 'creative director', 'WernerHerzog': 'Werner Herzog',
            'Promoterentertainment': 'promoter entertainment', 'HotelChelsea': 'Hotel Chelsea',
            'Nudephotography': 'nude photography', 'ChuckHagel': 'Chuck Hagel',
            'SamBrownback': 'Sam Brownback', 'TomCoburn': 'Tom Coburn', 
            'FrankLautenberg': 'Frank Lautenberg', 'ChuckSchumer': 'Chuck Schumer',
            'GreatRecession': 'Great recession', 
            'Financialcrisisof2007E280932008': 'Financial crisis (2007-2008)',
            'EconomyoftheUnitedStates': 'Economy of the U.S.', 'Balanceoftrade': 'Balance of trade',
            'Economicgrowth': 'Economic growth', 'Priceofoil': 'Price of oil',
            'Marketeconomics': 'Market economics', 'Balancedbudget': 'Balanced budget',
            'Termlimit': 'Term limit', 'FederalgovernmentoftheUnitedStates': 'Federal government of the U.S.',
            'UnitedStatesfederalbudget': 'U.S. federal budget', 'Billlaw': 'Bill law',
            'StatelegislatureUnitedStates': 'State legislature U.S.',
            'PresidencyofGeorgeWBush': 'Presidency of George Bush', 'lawmakers': 'law makers',
            'PatientProtectionandAffordableCareAct': '"Obama Care"',
            'GeneralServicesAdministration': 'General Services Administration',
            'OklahomaCity': 'Oklahoma City', '1993WorldTradeCenterbombing': 'World Trade Center bombing (1993)',
            'OklahomaCitybombing': 'Oklahoma City bombing', 'PortAuthorityBusTerminal': 'Port Authority Bus Terminal',
            'ColumbusCircle': 'Columbus Circle', 'Towerblock': 'Tower block',
            'WorldTradeCentersite': 'World Trade Center site', 'OneWorldTradeCenter': 'One World Trade Center',
            'CollapseoftheWorldTradeCenter': 'Collapse of the World Trade Center',
            'Arabworld': 'Arab world', 'IranE2809IraqWar': 'Iran & Iraq War',
            'IranianRevolution': 'Iranian revolution', 'Muslimworld': 'Muslim world',
            'DurandLine': 'Durand Line', 'SinaiPeninsula': 'Sinai Peninsula',
            'SwatDistrict': 'Swat District', 'Refugeecamp': 'Refugee camp',
            'Warcrime': 'War crime', 'ArmenianGenocide': 'Armenian genocide',
            'WorldWarI': 'World war I', 'TheHolocaust': 'The Holocaust',
            'SouthernLebanon': 'Southern Lebanon', 'Serbianlanguage': 'Serbian language',
            'Autonomousadministrativedivision': 'Autonomous administrative division',
            'NaziGerman': 'Nazi German', 'RobertMapplethorpe': 'Robert Mapplethorpe',
            'GuantC3A1namoBay': 'Guantanamo Bay', 'RussianEmpire': 'Russian empire',
            'One-partystate': 'One-party State', 'SovietUnion': 'Soviet Union',
            'PresidentofFrance': 'President of France', 'BritishEmpire': 'British Empire',
            'CoupdC3A9tat': 'coup d\'état', 'Doubleagent': 'Double agent',
            'FederalBureauofInvestigation': 'FBI', 'EastGermany': 'East Germany',
            'PaulBremer': 'Paul Bremer', '1953IraniancoupdC3A9tat': 'Iranian coup d\'état (1953)',
            'GamalAbdelNasser': 'Gamal Abdel Nasser', 'MohammadMosaddegh': 'Mohammad Mosaddegh',
            'UnitedStatesArmyReserve': 'U.S. ArmyReserve', 'Activeduty': 'Active duty',
            '7July2005Londonbombings': '7 July 2005 London bombings', 'TitleIX': 'Title IX',
            'ThePentagon': 'The Pentagon', 'DefenseIntelligenceAgency': 'Defense Intelligence Agency',
            'UnitedStatesDeparmentofjustice': 'U.S. Deparment of justice',
            'Intelligenceassessment': 'Intelligence assessment', 
            'UnitedStatesArmedForces': 'U.S. Armed Forces', 'Biologicalwarfare': 'Biological warfare',
            'Militarytechnology': 'Military technology', 'Nuclearweapon': 'Nuclear weapon',
            'Nuclearreactor': 'Nuclear reactor', 'Chemicalweapon': 'Chemical weapon',
            'Biologicalwarfare': 'Biological warfare', 'Nuclearproliferation': 'Nuclear proliferation',
            'NorthKorea': 'North Korea', 'Enricheduranium': 'Enriched uranium',
            'AdamPhilipspsychologist': 'Adam Philips psychologist', 'JeanineTesori': 'Jeanine Tesori',
            'ScholasticCorporation': 'Scholastic Corporation', 'dicaprio': 'di Caprio',
            'HyperionBooks': 'Hyperion Books', 'Sciencejournal': 'Science journal',
            'Yogajournal': 'Yoga journal', 'Counterpointpublisher': 'Counterpoint publisher',
            'HomerSimpson': 'Homer Simpson', 'Conflictresolution': 'Conflict resolution',
            'ConnieMack': 'Connie Mack', 'BrianFriel': 'Brian Friel', 'JohnMcGraw': 'John McGraw',
            'HughJohnsonwinewriter': 'Hugh Johnson winewriter', 
            'Sciencefiction': 'Science fiction', 'Shortstory': 'Short story',
            'ErnestHemingway': 'Ernest Hemingway', 'StephenKing': 'Stephen King',
            'AtomEgoyan': 'Atom Egoyan', 'EduardoMachado': 'Eduardo Machado',
            'MikeNewelldirector': 'Mike Newell director', 'Assistfootball': 'Assist football',
            'Capsport': 'Cap sport',

            'MassachusettsInstituteofTechnology': 'MIT',
            'ColumbiaUniversity': 'Columbia University', 
            'PrincetonUniversity': 'Princeton University',
            'NewYorkUniversity': 'New York University',
            'StanfordUniversity': 'Stanford University',
            'UniversityofCaliforniaBerkeley': 'University of California Berkeley',
            'UniversityofChicago': 'University of Chicago'
            }

# **Data exploration**

In [0]:
data = []
filenames = ['/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_1995_annotated_dandelion_stopwords', '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2000_annotated_dandelion_stopwords', '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2005_annotated_dandelion_stopwords', '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2010_annotated_dandelion_stopwords']

for filename in filenames:
  with open(filename, "r") as fifi:
    data = fifi.readlines()
    numberArticle = len(data)
    print("Numero di articoli:", numberArticle)
    meanNumWord = 0
    for article in data:
      meanNumWord = meanNumWord + len(article)
    meanNumWord = meanNumWord /numberArticle
    print("Numero di parole medio per articolo: ", int(meanNumWord))

# **Preprocessing**


In [0]:
# read file function
def file_get_texts(filename):
  data = []
  with open(filename, "r") as fifi:
    data = fifi.readlines()
  return data

# function to remove " 's " from words
def remove_stop_words(data):
  stops_data = []
  for article in data:
    stops_data.append(article.replace("'s", " "))

  return stops_data

# function to preprocess tokens
#operations: convert to lower case
def preprocessing_tokens(data):
  preprocess_data = []
  for article in data:
    preprocess_article = []
    for token in article:
      if 'DBRSTART' not in token and 'DBREND' not in token:
        preprocess_article.append(token.lower())
      else:
        preprocess_article.append(token)
    preprocess_data.append(preprocess_article)
  return preprocess_data

# read articles and extract tokens
def articles_get_contents(articles):
  data = list(map(lambda x : x.strip().split(), articles))
  return data

In [0]:
def merge_entities(data):
  '''
  Given a list of lists, where each sub-list is an article and each element of 
  list is a different word, the function merges the entities. It returns the 
  very same structure as the input data: list of lists.
  Parameters:
  data: list of lists. Each sub-list contains all words of an article
        every word is a different element of the sub-list.
  '''
  replacebad_chars = ["(", ")", "_", "&", "%", "!", "'", "+", "$", "@", ";", "*", " "]
  flattened_articles = [' '.join(x) for x in data]
  articles_merged_entities = list()
  for a in flattened_articles:
    fs = re.findall('DBRSTARTdbr:.+?DBREND|\w+', a)
    for char in replacebad_chars:
      fs = [x.replace(char, '') for x in fs]
    articles_merged_entities.append(fs)
  return articles_merged_entities

In [0]:
def find_entities(data):
  '''
  computes the list of unique and non-unique entities in a corpus.

  Parameters:
  data: list of lists. Each sub-list contains all words of an article
        every word is a different element of the sub-list.
  '''
  replacebad_chars = ["(", ")", "_", "&", "%", "!", "'", "+", "$", "@", ";", "*", " "]
  flattened_articles = [' '.join(x) for x in data]
  entities_set = set()
  not_unique = list()
  for a in flattened_articles:
    # the '?' is to avoid the greedy matching
    fs = re.findall('DBRSTARTdbr:.+?DBREND', a)
    for char in replacebad_chars:
      fs = [x.replace(char, '') for x in fs]
    entities_set.update(fs)
    not_unique += fs
  return entities_set, not_unique

In [0]:
# load and preprocess articles
file = '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_1995_annotated_dandelion_stopwords'
articles_1995 = file_get_texts(file)

file = '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2000_annotated_dandelion_stopwords'
articles_2000 = file_get_texts(file)

file = '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2005_annotated_dandelion_stopwords'
articles_2005 = file_get_texts(file)

file = '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2010_annotated_dandelion_stopwords'
articles_2010 = file_get_texts(file)

#file = '/content/gdrive/My Drive/ProgettoAI/TemporalNYData/text_2015_annotated_dandelion_stopwords'
#articles_2015 = file_get_texts(file)

# remove " 's "
print("Removing 's")
stops_articles_1995 = remove_stop_words(articles_1995)
stops_articles_2000 = remove_stop_words(articles_2000)
stops_articles_2005 = remove_stop_words(articles_2005)
stops_articles_2010 = remove_stop_words(articles_2010)
#stops_articles_2015 = remove_stop_words(articles_2015)

# extract tokens
print("Extracting tokens")
data_1995 = articles_get_contents(stops_articles_1995)
data_2000 = articles_get_contents(stops_articles_2000)
data_2005 = articles_get_contents(stops_articles_2005)
data_2010 = articles_get_contents(stops_articles_2010)
#data_2015 = articles_get_contents(stops_articles_2015)

# merge entities
print("Merging etities")
data_1995 = merge_entities(data_1995)
data_2000 = merge_entities(data_2000)
data_2005 = merge_entities(data_2005)
data_2010 = merge_entities(data_2010)
#data_2015 = merge_entities(data_2015)

print("Preprocessing tokens")
data_1995 = preprocessing_tokens(data_1995)
data_2000 = preprocessing_tokens(data_2000)
data_2005 = preprocessing_tokens(data_2005)
data_2010 = preprocessing_tokens(data_2010)

In [0]:
def save_article_to_file(articles, path):
  merged_article = [' '.join(x) for x in articles]
  with open(path, "w") as f:
    for a in merged_article:
      f.write(a)
      f.write(" ")

In [0]:
save_article_to_file(data_1995, "/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_1995.txt")
save_article_to_file(data_2000, "/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2000.txt")
save_article_to_file(data_2005, "/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2005.txt")
save_article_to_file(data_2010, "/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2010.txt")
#save_article_to_file(data_2015, "/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2015.txt")

file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_1995", 'wb')
pickle.dump(data_1995, file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2000", 'wb')
pickle.dump(data_2000, file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2005", 'wb')
pickle.dump(data_2005, file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2010", 'wb')
pickle.dump(data_2010, file)
file.close()

# **TWEC**

## **Training**

In [0]:
# create compass 
def createCompassFile():
  path = "/content/gdrive/My Drive/ProgettoAI/Merged_articles/"
  pathTWEC = '/content/gdrive/My Drive/ProgettoAI/twec-master/examples/training/'
  
  data = list()
  filenames = ["Merged_1995.txt", "Merged_2000.txt", "Merged_2005.txt", "Merged_2010.txt"]

  for filename in tqdm(filenames):
    with open(path + filename, "r") as reader:
      data.extend(reader.readlines())
  
  with open(pathTWEC + "compassNYT", "w") as out_file:
    out_file.write(str(data))
    
createCompassFile()

In [0]:
pathTWEC = '/content/gdrive/My Drive/ProgettoAI/twec-master/examples/training/'
  
# train the compass: the text is the concatenation of the text from the slices
# NB, ignoring words that occurs less than 5 times in the whole set of corpus
aligner = TWEC(size = embedding_size, siter = 10, diter = 10, workers = 2, min_count = 5)
aligner.train_compass(pathTWEC + "compassNYT", overwrite = True)

In [0]:
# train and dump different slices

print("Training slices.")
slice_1995 = aligner.train_slice('/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_1995.txt', save = True)
slice_2000 = aligner.train_slice('/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2000.txt', save = True)
slice_2005 = aligner.train_slice('/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2005.txt', save = True)
slice_2010 = aligner.train_slice('/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2010.txt', save = True)
#slice_2015 = aligner.train_slice('/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2015.txt' , save = True)

## **Loading models**

In [0]:
# load W2V slices
model1995 = Word2Vec.load("model/Merged_1995.model")
model2000 = Word2Vec.load("model/Merged_2000.model")
model2005 = Word2Vec.load("model/Merged_2005.model")
model2010 = Word2Vec.load("model/Merged_2010.model")
#model2015 = Word2Vec.load("model/Merged_2015.model")

In [0]:
# function to compute similarity between two entity embeddings
def calculateSimiliraty(word1, word2):
  return 1 - cosine(word1, word2)

## **Similarities**

In [0]:
def get_trend_single_slice(entity_name, n_similar, models, year):
  if entity_name in models[year]:
    most_similar = []

    # save top n similar entities of all slices
    for entity in models[year].most_similar(entity_name)[:n_similar]:
      most_similar.append(entity[0])

    # compute similarity for each entity of each slice 
    # dict {entity: [sim_1995, sim_2000, sim_2005, sim_2010]}

    similarities = {}
    for entity in most_similar:
      similarities[entity] = []
      if entity in models['1995'] and entity_name in models['1995']:
        similarities[entity].append(calculateSimiliraty(models['1995'][entity], models['1995'][entity_name]))
      else:
        similarities[entity].append(0)

      if entity in models['2000'] and entity_name in models['2000']:
        similarities[entity].append(calculateSimiliraty(models['2000'][entity], models['2000'][entity_name]))
      else:
        similarities[entity].append(0)

      if entity in models['2005'] and entity_name in models['2005']:
        similarities[entity].append(calculateSimiliraty(models['2005'][entity], models['2005'][entity_name]))
      else:
        similarities[entity].append(0)

      if entity in models['2010'] and entity_name in models['2010']:
        similarities[entity].append(calculateSimiliraty(models['2010'][entity], models['2010'][entity_name]))
      else:
        similarities[entity].append(0)

    return similarities
  else:
    print("No entity found in slice " + year + ".")      
    return {}

def subplot(similar, ax, year, legend_position, legend_location):
  groups = ("1995", "2000", "2005", "2010")

  # subplot trend of similarities of each slice 
  lines = []
  for entity in similar:
    '''
    labelName = entity
    if labelName in nameDict:
      labelName = nameDict[labelName]
    '''
    line, = ax.plot(similar[entity], label = entity)
    lines.append(line)
  
  ax.set_title('Slice ' + year)
  ax.set_ylim(bottom = 0, top = 1)

  plt.sca(ax)
  plt.xticks((0, 1, 2, 3), groups)
  
  #plt.subplots_adjust(left = 0.1, right = 0.11)
  '''
  left  = 0.125,  # the left side of the subplots of the figure
  right = 0.9,    # the right side of the subplots of the figure
  bottom = 1,   # the bottom of the subplots of the figure
  top = 0.9,      # the top of the subplots of the figure
  wspace = 0.2,   # the amount of width reserved for blank space between subplots
  hspace = 0.3)
  '''
  
  names = []
  for entity in similar.keys():
    entity = entity.replace("DBRSTARTdbr:", "").replace("DBREND", "")
    if entity in nameDict:
      entity = nameDict[entity]
    names.append(entity)

  # Shrink current axis by 20%
  box = ax.get_position()
  ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

  # Put a legend to the right of the current axis
  l = ax.legend(lines, names, 
            loc = legend_location, 
            bbox_to_anchor = legend_position)

  #bbox = l.get_window_extent() 
  #print(bbox.width,bbox.height)

def plot_trend_subplots(similarities, entity, subdirectory):
  # create four polar axes, and access them through the returned array
  trimmedEntity = entity_name.replace("DBRSTARTdbr:", "").replace("DBREND", "")
  fig, axes = plt.subplots(2, 2, figsize = (12, 8), 
                           sharex = True, sharey = True, dpi = 300)
  if trimmedEntity in nameDict:
    trimmedEntity = nameDict[trimmedEntity]
  plt.suptitle("Similarity trend of {}".format(trimmedEntity), fontsize = 16)
  for i, row in enumerate(axes):
    for j, cell in enumerate(row):
        if i == len(axes) - 1:
            cell.set_xlabel("Time")
        if j == 0:
            cell.set_ylabel("Similarity")
  subplot(similarities['1995'], axes[0, 0], '1995', (-0.2, 0.7), 0)
  subplot(similarities['2000'], axes[0, 1], '2000', (1.1, 0.5), 6)
  subplot(similarities['2005'], axes[1, 0], '2005', (-0.2, 0.7), 0)
  subplot(similarities['2010'], axes[1, 1], '2010', (1.1, 0.5), 6)
  
  try:
    os.mkdir("./images/trend/{}".format(subdirectory))
  except:
    print("Directory already exist.")
  plt.savefig("./images/trend/{}/{}.png".format(subdirectory, trimmedEntity), bbox_inches='tight')
  plt.close()

In [0]:
entity_name = "DBRSTARTdbr:Al-QaedaDBREND"
n_similar = 3

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'test')

# **PCA**

In [0]:
def calculate_pca(model):
  all_vectors = []
  vocaboli = []
  for key in model.wv.vocab:  # search for all words in the vocabulary and add the associated representation
    vector = model.wv[key]
    all_vectors.append(vector)
    vocaboli.append(key)

  pca = decomposition.PCA(n_components = 2, random_state = 42)
  pca.fit(all_vectors)
  pca_vectors = pca.transform(all_vectors)
  pca_dictionary = dict(zip(vocaboli, pca_vectors))

  return pca_dictionary


pca_1995 = calculate_pca(model1995)
pca_2000 = calculate_pca(model2000)
pca_2005 = calculate_pca(model2005)
pca_2010 = calculate_pca(model2010)

In [0]:
def plot_evolution_NN_entity(entity, pcas, models, colors, subdirectory, 
                             n_subplots = 4, n_most_similar = 5):
  '''
  Parameters:
  entity: string containing the name of the entity to explore.
  pcas: dict, key is the year and value is the actual pca of the alignment
  models: dict key is the year and values is the model
  n_subplots: number of subplots to be generated in the same figure
  n_most_similar: number of most similar to find
  '''
  xs = dict() # key is the year, value is the list o x coord
  ys = dict() # key is the year, value is the list o y coord
  names = dict() # key is the year, value is the list o names to annotate
  toPlot = dict() # key is the year, value is the list of flag of ax to plot,
                  # in case of some exception thrown
  for k in pcas.keys():
    # The first element of the lists is always the entity coords
    try:
      x, y = pcas[k][entity]
    except:
      toPlot[k] = False
      continue
    toPlot[k] = True
    xs[k] = [x]
    ys[k] = [y]
    names[k] = [entity]

    # Adding the most similar
    nameMostSimilar = [e[0] for e in models[k].most_similar(entity)[:n_most_similar]]

    for name in nameMostSimilar:
      x, y = pcas[k][name]
      xs[k].append(x)
      ys[k].append(y)
    names[k] += nameMostSimilar

    # cleaning the annotations from entities
    names[k] = [e.replace("DBRSTARTdbr:", '') for e in names[k]]
    names[k] = [e.replace("DBREND", '') for e in names[k]]

  # Plotting
  fig, axes = plt.subplots(nrows = 2, ncols = n_subplots // 2 + n_subplots % 2, 
                           sharex = False, sharey = False, 
                           figsize = (8, 6), dpi = 300)
  # if we share the axes, the points will collapse showing that the entity 
  # semantic may have shifted (as in case of AppleInc) but messing a lot
  # the annotation
  trimmedEntity = entity.replace("DBRSTARTdbr:", '').replace("DBREND", '')
  if trimmedEntity in nameDict:
    trimmedEntity = nameDict[trimmedEntity]
  plt.suptitle('Evolution of {}'.format(trimmedEntity), fontsize = 16)
  for ax, k, c in zip(axes.reshape(-1), toPlot.keys(), colors[:n_subplots]):
    ax.set_title(str(k), fontsize = 12)
    if toPlot[k]:
      ax.scatter(xs[k][0], ys[k][0], s = 100, color = c, marker = 'x')
      ax.scatter(xs[k][1:], ys[k][1:], s = 40, color = c)
      annotations = [ax.text(x, y, nameDict[name] if name in nameDict else name, fontsize = 10) for x, y, name in zip(xs[k][1:], ys[k][1:], names[k][1:])]
      adjust_text(annotations, x = xs[k], y = ys[k], ax = ax)
    ax.tick_params(axis = 'both', which = 'both', bottom = False, top = False,
                   labelbottom = False, left = False, labelleft = False)

  try:
    os.mkdir("./images/evolution/{}".format(subdirectory))
  except:
    pass
  plt.savefig('./images/evolution/{}/{}.png'.format(subdirectory, trimmedEntity))
  plt.close()

In [0]:
pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
plot_evolution_NN_entity('DBRSTARTdbr:GoogleDBREND', pcas, models, colors, subdirectory = 'test')
plot_evolution_NN_entity('DBRSTARTdbr:AppleIncDBREND', pcas, models, colors, subdirectory = 'test')

In [0]:
def plotPCAovertime(entities, plotname, colors, subdirectory):
  # entities: list of words to plot 
  data_1995 = list()
  data_2000 = list()
  data_2005 = list()
  data_2010 = list()

  markers = ["o", "X", "s", "d", "P", "v", "|", "*", "_"]
  groups = ("1995", "2000", "2005", "2010")
  names = list()
  placeholder = np.array(["?", "?"])

  for entity in entities:
    if entity in pca_1995.keys():
      data_1995 += [pca_1995[entity]] # add pca values
    else:
      print("Entity " + str(entity) + " missing in 1995")
      data_1995 += [placeholder]    # add placeholder for missing value
    if entity in pca_2000.keys():
      data_2000 += [pca_2000[entity]]
    else:
      print("Entity " + str(entity) + "  missing in 2000")
      data_2000 += [placeholder] 
    if entity in pca_2005.keys():
      data_2005 += [pca_2005[entity]]
    else:
      print("Entity " + str(entity) + " missing in 2005")
      data_2005 += [placeholder] 
    if entity in pca_2010.keys():
      data_2010 += [pca_2010[entity]]
    else:
      print("Entity " + str(entity) + " missing in 2010")
      data_2010 += [placeholder] 
    
    name = entity                             # get name and remove annotation
    name = name.replace("DBRSTARTdbr:", "")
    name = name.replace("DBREND", "")
    names.append(name)

  fig = plt.figure(figsize = (8, 6), dpi = 300)
  ax = fig.add_subplot(1, 1, 1)
  
  dataScatter = (data_1995, data_2000, data_2005, data_2010)

  for dataScatter, color in zip(dataScatter, colors):
    i = 0
    for point in dataScatter:
      name = names[i]
      x, y = point
      if x != "?" and y != "?":   # check for missing value
        ax.scatter(x, y, alpha = 0.9, c = color, edgecolors = 'none', s = 50, 
                   marker = markers[i], label = name)
        #ax.annotate(name, (x, y))
      i = i + 1 

  plt.title('Evolution of entities over time', fontsize = 18)
  plt.xlabel("PCA x value", fontsize = 14)
  plt.ylabel("PCA y value", fontsize = 14)
  plt.xticks(fontsize = 8)
  plt.yticks(fontsize = 8)

  # add legend
  patches = list()
  patch_1995 = mpatches.Patch(color = colors[0], label = '1995')
  patch_2000 = mpatches.Patch(color = colors[1], label = '2000')
  patch_2005 = mpatches.Patch(color = colors[2], label = '2005')
  patch_2010 = mpatches.Patch(color = colors[3], label = '2010')

  patches = [patch_1995, patch_2000, patch_2005, patch_2010]

  i = 0
  for n in names:
    name = n
    if name in nameDict:
      name = nameDict[name]
    tmp, = plt.plot([], [], marker = markers[i], ls = "", label = name, 
                    mfc = 'none', alpha = .9, color = 'Black', markersize = 9)
    patches.append(tmp)
    i = i + 1 
    
  # Shrink current axis by 20%
  box = ax.get_position()
  ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

  # Put a legend to the right of the current axis
  ax.legend(handles = patches, loc = 'center left', bbox_to_anchor = (1, 0.5))
  try:
    os.mkdir("./images/pcaovertime/{}".format(subdirectory))
  except:
    pass
  plt.savefig("./images/pcaovertime/{}/{}.png".format(subdirectory, plotname), 
              bbox_inches = 'tight')
  plt.close()

# Test
entities = ['DBRSTARTdbr:MicrosoftDBREND', 'DBRSTARTdbr:AppleIncDBREND', 
            'DBRSTARTdbr:IBMDBREND', 'DBRSTARTdbr:GoogleDBREND']
plotPCAovertime(entities, "Apple", colors, subdirectory = 'test')

# **Yugoslav Wars (1991 - 2001)**

In [0]:
'''ENTITA':
DBRSTARTdbr:SerbiaDBREND
DBRSTARTdbr:CroatiaDBREND
DBRSTARTdbr:KosovoDBREND             
DBRSTARTdbr:WorldWarIIDBREND          
DBRSTARTdbr:RussiaDBREND              
DBRSTARTdbr:NATODBREND
DBRSTARTdbr:BosniaandHerzegovinaDBREND


#Aggiunti
DBRSTARTdbr:YugoslaviaDBREND
DBRSTARTdbr:BosnianWarDBREND
DBRSTARTdbr:CivilwarDBREND
DBRSTARTdbr:NazionalismDBREND
DBRSTARTdbr:IndependenceDBREND       

DBRSTARTdbr:SloveniaDBREND              
DBRSTARTdbr:RepublicofMacedoniaDBREND  

DBRSTARTdbr:SocialismDBREND             

#Parole importanti
#  war
#  communism   DBRSTARTdbr:CommunismDBREND
#  Genocide    DBRSTARTdbr:GenocideDBREND
'''

# CIVIL WAR  
entity_name = 'DBRSTARTdbr:CivilwarDBREND'     
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'YugoslavWars')


# GENOCIDIO 
entity_name = 'DBRSTARTdbr:GenocideDBREND'     
n_similar = 5                                 

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'YugoslavWars')



entities = ['DBRSTARTdbr:SerbiaDBREND', 'DBRSTARTdbr:KosovoDBREND', 'DBRSTARTdbr:BosniaandHerzegovinaDBREND', 'DBRSTARTdbr:YugoslaviaDBREND', 'DBRSTARTdbr:CroatiaDBREND','DBRSTARTdbr:CommunismDBREND', 'DBRSTARTdbr:BosnianWarDBREND', 'DBRSTARTdbr:NATODBREND', 'DBRSTARTdbr:CivilwarDBREND']
plotPCAovertime(entities, "comunism", colors, subdirectory = 'YugoslavWars')



entities = ['DBRSTARTdbr:SerbiaDBREND', 'DBRSTARTdbr:IndependenceDBREND', 'DBRSTARTdbr:BosniaandHerzegovinaDBREND', 'DBRSTARTdbr:SocialismDBREND', 'DBRSTARTdbr:BosnianWarDBREND', 'DBRSTARTdbr:CivilwarDBREND', 'DBRSTARTdbr:NationalismDBREND', 'war', 'DBRSTARTdbr:GenocideDBREND']
plotPCAovertime(entities, "war", colors, subdirectory = 'YugoslavWars')


pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}


plot_evolution_NN_entity('war', pcas, models, colors, subdirectory = 'YugoslavWars')
plot_evolution_NN_entity('DBRSTARTdbr:BosnianWarDBREND', pcas, models, colors, subdirectory = 'YugoslavWars')
plot_evolution_NN_entity('DBRSTARTdbr:CivilwarDBREND', pcas, models, colors, subdirectory = 'YugoslavWars')
plot_evolution_NN_entity('DBRSTARTdbr:NationalismDBREND', pcas, models, colors, subdirectory = 'YugoslavWars')
plot_evolution_NN_entity('DBRSTARTdbr:GenocideDBREND', pcas, models, colors, subdirectory = 'YugoslavWars')
plot_evolution_NN_entity('DBRSTARTdbr:YugoslaviaDBREND', pcas, models, colors, subdirectory = 'YugoslavWars')


# **Star wars**

In [0]:
# 
'''ENTITA':
DBRSTARTdbr:StarWarsDBREND o 
DBRSTARTdbr:StarWarsfilmDBREND # meglio usare starwars
DBRSTARTdbr:StarTrekDBREND

DBRSTARTdbr:SpecialeffectDBREND


personaggi non ci sono in alcuni anni
DBRSTARTdbr:LukeSkywalkerDBREND
DBRSTARTdbr:Obi-WanKenobiDBREND 
DBRSTARTdbr:PrincessLeiaDBREND 
DBRSTARTdbr:HanSoloDBREND  

# Parole importanti
Star wars
Phantom menace 
attack of clones
revenge of the sith
star trek
film
trilogy
'''


entity_name = 'DBRSTARTdbr:StarWarsDBREND'
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'StarWars')



entities = ['DBRSTARTdbr:StarWarsDBREND', 'DBRSTARTdbr:SpecialeffectDBREND', 'DBRSTARTdbr:StarTrekDBREND','film', 'trilogy','DBRSTARTdbr:HanSoloDBREND']
plotPCAovertime(entities, "comunism", colors, subdirectory = 'StarWars')


pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}


plot_evolution_NN_entity('DBRSTARTdbr:StarWarsDBREND', pcas, models, colors, subdirectory = 'StarWars')
plot_evolution_NN_entity('DBRSTARTdbr:SpecialeffectDBREND', pcas, models, colors, subdirectory = 'StarWars')
plot_evolution_NN_entity('DBRSTARTdbr:StarTrekDBREND', pcas, models, colors, subdirectory = 'StarWars')




# **Call of Duty**

In [0]:
# 
'''
dbr:CallofDuty
violence
videogames
terrorism
war

DBRSTARTdbr:VideoGameDBREND non c'è 
DBRSTARTdbr:GameDBREND
'''


entity_name = 'DBRSTARTdbr:CallofDutyDBREND'
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'CallofDuty')



entities = ['DBRSTARTdbr:CallofDutyDBREND','violence', 'DBRSTARTdbr:GameDBREND', 'war','terrorism', 'video', 'games']
plotPCAovertime(entities, "videogames", colors, subdirectory = 'CallofDuty')


pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}

plot_evolution_NN_entity('DBRSTARTdbr:CallofDutyDBREND', pcas, models, colors, subdirectory = 'CallofDuty')
plot_evolution_NN_entity('violence', pcas, models, colors, subdirectory = 'CallofDuty')
plot_evolution_NN_entity('DBRSTARTdbr:GameDBREND', pcas, models, colors, subdirectory = 'CallofDuty')
plot_evolution_NN_entity('war', pcas, models, colors, subdirectory = 'CallofDuty')
plot_evolution_NN_entity('terrorism', pcas, models, colors, subdirectory = 'CallofDuty')

# **Eventi sportivi**


In [0]:
# 
'''
#squadre
'DBRSTARTdbr:AmericanFootballTeamDBREND', 
'DBRSTARTdbr:ItalynationalfootballteamDBREND',  
'DBRSTARTdbr:FrancenationalfootballteamDBREND',  
'DBRSTARTdbr:GermanynationalfootballteamDBREND',  
'DBRSTARTdbr:SpainnationalfootballteamDBREND',  
'DBRSTARTdbr:NetherlandsnationalfootballteamDBREND',  
'DBRSTARTdbr:BrazilnationalfootballteamDBREND' 

#paesi ospitanti
# usa 94            DBRSTARTdbr:1994FIFAWorldCupDBREND     
# francia 98        DBRSTARTdbr:1998FIFAWorldCupDBREND
# corea giappone    DBRSTARTdbr:2002FIFAWorldCupDBREND
# germania          DBRSTARTdbr:2006FIFAWorldCupDBREND
# sud africa        DBRSTARTdbr:2010FIFAWorldCupDBREND

'''

entity_name = 'DBRSTARTdbr:AmericanFootballTeamDBREND'
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'FIFA_World_Cup')



entities = ['DBRSTARTdbr:AmericanFootballTeamDBREND', 
'DBRSTARTdbr:ItalynationalfootballteamDBREND',  
'DBRSTARTdbr:FrancenationalfootballteamDBREND',  
'DBRSTARTdbr:GermanynationalfootballteamDBREND',  
'DBRSTARTdbr:SpainnationalfootballteamDBREND',  
'DBRSTARTdbr:NetherlandsnationalfootballteamDBREND',  
'DBRSTARTdbr:BrazilnationalfootballteamDBREND']
plotPCAovertime(entities, "football", colors, subdirectory = 'WorlCup')


'''
# Olympic_Games
entity_name = 'DBRSTARTdbr:OlympicGamesDBREND'
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'Olympic_Games')

'''

# **Guerra Iraq (2003 - 2011)**

In [0]:
'''Entità:

DBRSTARTdbr:GeorgeWBushDBREND
DBRSTARTdbr:SaddamHusseinDBREND
DBRSTARTdbr:NATODBREND
DBRSTARTdbr:Al-QaedaDBREND
DBRSTARTdbr:WorldTradeCenter1973E280932001DBREND
DBRSTARTdbr:IraqDBREND
DBRSTARTdbr:UnitedStatesDBREND
DBRSTARTdbr:CentralIntelligenceAgencyDBREND
DBRSTARTdbr:BillClintonDBREND
DBRSTARTdbr:BarackObamaDBREND
DBRSTARTdbr:PetroleumDBREND

#Parole importanti
# war
# terrorism
# oil 

'''

keywords = ["DBRSTARTdbr:GeorgeWBushDBREND", "DBRSTARTdbr:SaddamHusseinDBREND",
            "DBRSTARTdbr:NATODBREND", "DBRSTARTdbr:Al-QaedaDBREND",
            "DBRSTARTdbr:WorldTradeCenter1973E280932001DBREND", 
            "DBRSTARTdbr:IraqDBREND", "DBRSTARTdbr:UnitedStatesDBREND", 
            "DBRSTARTdbr:CentralIntelligenceAgencyDBREND", 
            "DBRSTARTdbr:BillClintonDBREND", "DBRSTARTdbr:BarackObamaDBREND",
            "DBRSTARTdbr:PetroleumDBREND", "war", "terrorism", "oil"]

In [0]:
# Plotting trend subplots
for kw in tqdm(keywords):
  entity_name = kw
  n_similar = 5

  models = {}
  models['1995'] = model1995
  models['2000'] = model2000
  models['2005'] = model2005
  models['2010'] = model2010

  similarities = {}

  similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
  similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
  similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
  similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

  plot_trend_subplots(similarities, entity_name, subdirectory = 'IraqWar')

In [0]:
# plotting evolution
for kw in tqdm(keywords):
  pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
  models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
  plot_evolution_NN_entity(kw, pcas, models, colors, subdirectory = 'IraqWar')

In [0]:
# plotting overtime
chiefs = ["DBRSTARTdbr:GeorgeWBushDBREND", "DBRSTARTdbr:SaddamHusseinDBREND",
          "DBRSTARTdbr:Al-QaedaDBREND",  "DBRSTARTdbr:BillClintonDBREND", 
          "DBRSTARTdbr:PetroleumDBREND", "war", "terrorism"]
plotPCAovertime(chiefs, "IraqWar-Chiefs", colors, subdirectory = 'IraqWar')

nine_eleven = ["DBRSTARTdbr:SaddamHusseinDBREND",
               "DBRSTARTdbr:Al-QaedaDBREND", "DBRSTARTdbr:WorldTradeCenter1973E280932001DBREND", 
               "DBRSTARTdbr:CentralIntelligenceAgencyDBREND", "terrorism"]
plotPCAovertime(nine_eleven, "IraqWar-9_11", colors, subdirectory = 'IraqWar')

# **Economical crisis (2008)**

In [0]:
'''Entità:
DBRSTARTdbr:LehmanBrothersDBREND
DBRSTARTdbr:HenryPaulsonDBREND   no 2000
DBRSTARTdbr:UnitedStatesCongressDBREND
DBRSTARTdbr:StockmarketDBREND
DBRSTARTdbr:StockDBREND
DBRSTARTdbr:MarketeconomicsDBREND
DBRSTARTdbr:FinancialmarketDBREND

#Parole di interesse
#economy
#crisis
#debt
#congress
#stock market (non la trovo)
#stock
#stocks
#Soros
'''
folder = 'economical_crisis'

In [0]:
# PCA over time plots

entities = ['economy', 'crisis', 'DBRSTARTdbr:LehmanBrothersDBREND', 'DBRSTARTdbr:HenryPaulsonDBREND']
plotPCAovertime(entities, folder, colors, folder)

In [0]:
entities = ['dbr:LehmanBrothers', 'dbr:HenryPaulson', 'dbr:UnitedStatesCongress', 'dbr:Stockmarket', 'dbr:Stock', 'dbr:Marketeconomics', 'dbr:Financialmarket']
for i in range(len(entities)):
  entities[i] = "DBRSTART" + entities[i] + 'DBREND'

words = ['economy', 'crisis', 'debt', 'congress', 'stock', 'Soros']

In [0]:
# Plot evolution

pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}

for entity in entities:
  plot_evolution_NN_entity(entity, pcas, models, colors, subdirectory = folder)

for word in words:
  plot_evolution_NN_entity(word, pcas, models, colors, subdirectory = folder)

In [0]:
# Plotting trend subplots
for kw in tqdm(entities):
  entity_name = kw
  n_similar = 5

  models = {}
  models['1995'] = model1995
  models['2000'] = model2000
  models['2005'] = model2005
  models['2010'] = model2010

  similarities = {}

  similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
  similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
  similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
  similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

  plot_trend_subplots(similarities, entity_name, subdirectory = 'economical_crisis')

# **Russia**

In [0]:
'''Entità:
DBRSTARTdbr:VladimirPutinDBREND
DBRSTARTdbr:BillClintonDBREND
DBRSTARTdbr:KGBDBREND
DBRSTARTdbr:CentralIntelligenceAgencyDBREND
DBRSTARTdbr:FederalSecurityServiceDBREND
DBRSTARTdbr:ChechnyaDBREND
DBRSTARTdbr:CommunismDBREND
DBRSTARTdbr:CrimeaDBREND
DBRSTARTdbr:CrimeanWarDBREND
DBRSTARTdbr:DmitryMedvedevDBREND
#Parole di interesse
#comunism #QUESTA PAROLA MANCA ma c'è entità 
#war
nuclear
'''
keywords = ["DBRSTARTdbr:VladimirPutinDBREND", "DBRSTARTdbr:BillClintonDBREND",
            "DBRSTARTdbr:KGBDBREND", "DBRSTARTdbr:CentralIntelligenceAgencyDBREND",
            "DBRSTARTdbr:FederalSecurityServiceDBREND", "DBRSTARTdbr:ChechnyaDBREND",
            "DBRSTARTdbr:CommunismDBREND", "DBRSTARTdbr:CrimeaDBREND", 
            "DBRSTARTdbr:CrimeanWarDBREND", "war", "nuclear", 
            "DBRSTARTdbr:DmitryMedvedevDBREND"]

In [0]:
# Plotting trend subplots
for kw in tqdm(keywords):
  entity_name = kw
  n_similar = 5

  models = {}
  models['1995'] = model1995
  models['2000'] = model2000
  models['2005'] = model2005
  models['2010'] = model2010

  similarities = {}

  similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
  similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
  similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
  similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

  plot_trend_subplots(similarities, entity_name, subdirectory = 'Russia')

In [0]:
# plotting evolution
for kw in tqdm(keywords):
  pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
  models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
  plot_evolution_NN_entity(kw, pcas, models, colors, subdirectory = 'Russia')

In [0]:
# plotting overtime
chiefs_war = ["DBRSTARTdbr:VladimirPutinDBREND", "DBRSTARTdbr:BillClintonDBREND",
              "war", "nuclear", "DBRSTARTdbr:DmitryMedvedevDBREND"]
plotPCAovertime(chiefs_war, "Russia-Chiefs & war", colors, subdirectory = 'Russia')

agencies = ["DBRSTARTdbr:KGBDBREND", "DBRSTARTdbr:CentralIntelligenceAgencyDBREND",
            "DBRSTARTdbr:FederalSecurityServiceDBREND", "DBRSTARTdbr:ChechnyaDBREND",
            "nuclear"]
plotPCAovertime(agencies, "Russia-agencies", colors, subdirectory = 'Russia')

# **Artificial Intelligence**

In [0]:
'''
Entità
DBRSTARTdbr:GoogleDBREND
DBRSTARTdbr:FacebookDBREND
DBRSTARTdbr:MassachusettsInstituteofTechnologyDBREND
DBRSTARTdbr:ArtificialneuralnetworkDBREND
DBRSTARTdbr:BiologicalneuralnetworkDBREND (c'è nei file ma non nei modelli)
DBRSTARTdbr:GenomicsDBREND

DBRSTARTdbr:ComputerprogrammingDBREND
DBRSTARTdbr:OpenInventionNetworkDBREND

#Parole di interesse
'''
folder = 'artificial_intelligence'

In [0]:
# PCA over time plots

entities = ['Google', 'Facebook', 'MassachusettsInstituteofTechnology', 'Artificialneuralnetwork', 'Genomics', 'Computerprogramming']
for i in range(len(entities)):
  entities[i] = "DBRSTARTdbr:" + entities[i] + 'DBREND'

plotPCAovertime(entities, folder, colors, folder)

In [0]:
entities = ['Google', 'Facebook', 'MassachusettsInstituteofTechnology', 'Artificialneuralnetwork', 'Genomics', 'Computerprogramming']
for i in range(len(entities)):
  entities[i] = "DBRSTARTdbr:" + entities[i] + 'DBREND'


In [0]:
# Plot evolution

pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}

for entity in entities:
  plot_evolution_NN_entity(entity, pcas, models, colors, subdirectory = folder)

for word in words:
  plot_evolution_NN_entity(word, pcas, models, colors, subdirectory = folder)

In [0]:
# Plotting trend subplots
for kw in tqdm(entities):
  entity_name = kw
  n_similar = 5

  models = {}
  models['1995'] = model1995
  models['2000'] = model2000
  models['2005'] = model2005
  models['2010'] = model2010

  similarities = {}

  similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
  similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
  similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
  similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

  plot_trend_subplots(similarities, entity_name, subdirectory = 'artificial_intelligence')

# **Harry Potter**

In [0]:
'''
Entità
DBRSTARTdbr:JKRowlingDBREND
DBRSTARTdbr:HarryPotterDBREND
DBRSTARTdbr:LordVoldemortDBREND

Parole di interesse

'''

folder = 'harry_potter'

In [0]:
# PCA over time plots

entities = ['JKRowling', 'HarryPotter', 'LordVoldemort']
for i in range(len(entities)):
  entities[i] = "DBRSTARTdbr:" + entities[i] + 'DBREND'

plotPCAovertime(entities, folder, colors, folder)

In [0]:
entities = ['JKRowling', 'HarryPotter', 'LordVoldemort']
for i in range(len(entities)):
  entities[i] = "DBRSTARTdbr:" + entities[i] + 'DBREND'

words = []

In [0]:
word = 'DBRSTARTdbr:LordVoldemortDBREND'
if word in models['1995']: 
  print(models['1995'].most_similar(word)[:5])
if word in models['2000']: 
  print(models['2000'].most_similar(word)[:5])
if word in models['2005']: 
  print(models['2005'].most_similar(word)[:5])
if word in models['2010']: 
  print(models['2010'].most_similar(word)[:5])

In [0]:
# Plot evolution

pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}

for entity in entities:
  plot_evolution_NN_entity(entity, pcas, models, colors, subdirectory = folder)

for word in words:
  plot_evolution_NN_entity(word, pcas, models, colors, subdirectory = folder)

In [0]:
# Plotting trend subplots
for kw in tqdm(entities):
  entity_name = kw
  n_similar = 5

  models = {}
  models['1995'] = model1995
  models['2000'] = model2000
  models['2005'] = model2005
  models['2010'] = model2010

  similarities = {}

  similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
  similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
  similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
  similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

  plot_trend_subplots(similarities, entity_name, subdirectory = 'harry_potter')

# **More analysis**

# Occurancies Functions

In [0]:
# count occurancies in evry year and return also the total count
def countOccorrunces(word):
  filenames = ['/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_1995.txt', '/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2000.txt', '/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2005.txt', '/content/gdrive/My Drive/ProgettoAI/Merged_articles/Merged_2010.txt']
  counts = list()
  for file in filenames:
    data = []
    with open(file, "r") as fifi:
      data = fifi.readlines()
      counts.append(data[0].count(word))
  return counts, sum(counts)


# print occurancies calculate from the function "countOccorrunces"
def printOccurancies(entities):
  for wordToCount in entities:
    counts, total = countOccorrunces(wordToCount)
    print("Occorrences over years " + wordToCount + ": ", counts)
    print("Total occurances " + wordToCount + ": ", total)
    print()


# calculate cosine similarity for word and list of entity (entities) over years
def calculateSimilarityOverYears(word, entities):
  similarity= {}
  for entity in entities:
    tmp = []
    if entity in model1995:
      tmp.append(round(calculateSimiliraty(model1995[word], model1995[entity]), 3))
    else:
        tmp.append(0)
    if entity in model2000:
      tmp.append(round(calculateSimiliraty(model2000[word], model2000[entity]), 3))
    else:
        tmp.append(0)
    if entity in model2005:
      tmp.append(round(calculateSimiliraty(model2005[word], model2005[entity]), 3))
    else:
        tmp.append(0)
    if entity in model2010:
      tmp.append(round(calculateSimiliraty(model2010[word], model2010[entity]), 3))
    else:
        tmp.append(0)
    similarity[entity] = tmp

  return similarity

In [0]:

pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
# frequency of Yugoslavia wars
print("frequenza parole guerra Yugoslava")
entities = ['DBRSTARTdbr:CivilwarDBREND', 'DBRSTARTdbr:BosnianWarDBREND','DBRSTARTdbr:NationalismDBREND','DBRSTARTdbr:GenocideDBREND','DBRSTARTdbr:YugoslaviaDBREND','DBRSTARTdbr:NATODBREND']
printOccurancies(entities)


# Frequency of obama entity
print("frequenza Obama: ", countOccorrunces('DBRSTARTdbr:BarackObamaDBREND') )


print("frequenza  WorldTradeCenter ", countOccorrunces('WorldTradeCenter1973E280932001'))


# Confronto petrolio e guerra 
print("Confronto petrolio e guerra: ")
word = 'DBRSTARTdbr:PetroleumDBREND'
entities = ['war', 'DBRSTARTdbr:IraqDBREND', 'DBRSTARTdbr:IranDBREND']
results = calculateSimilarityOverYears(word, entities)
print(results)
print()



# Similarità KGB con entità Iraq (Bin Laden, nuclear, saddam, Iraq ...)
print("KGB con Iraq (Bin Laden, nuclear, saddam, Iraq) : ")
word = 'DBRSTARTdbr:KGBDBREND'
entities = ["nuclear", "DBRSTARTdbr:IraqDBREND", "DBRSTARTdbr:IranDBREND", "DBRSTARTdbr:SaddamHusseinDBREND", "DBRSTARTdbr:OsamabinLadenDBREND"]
results = calculateSimilarityOverYears(word, entities)
print(results)
print()

plotPCAovertime(entities, "richiestaPapettiRussia", colors, subdirectory = 'Papetti')


# Nato con nazioni tipo: Bosnia, Iraq e altri -> overtime
entities = ['DBRSTARTdbr:KosovoDBREND', 'DBRSTARTdbr:BosniaandHerzegovinaDBREND', 'DBRSTARTdbr:YugoslaviaDBREND', 'DBRSTARTdbr:IraqDBREND','DBRSTARTdbr:IranDBREND', 'DBRSTARTdbr:NATODBREND']
plotPCAovertime(entities, "richiestaPape1", colors, subdirectory = 'Papetti')

# Overtime -> Bosnia e comunismo
entities = ['DBRSTARTdbr:BosniaandHerzegovinaDBREND', 'DBRSTARTdbr:CommunismDBREND']
plotPCAovertime(entities, "richiestaPape2", colors, subdirectory = 'Papetti')

# Overtime ->  war, genocidio, nazionalismo e civilwar
entities = ['war', 'DBRSTARTdbr:CivilwarDBREND', 'DBRSTARTdbr:NationalismDBREND', 'DBRSTARTdbr:GenocideDBREND']
plotPCAovertime(entities, "richiestaPape3", colors, subdirectory = 'Papetti')



# indagare cosa c'è vicino alla rowling  DBRSTARTdbr:JKRowlingDBREND DBRSTARTdbr:HarryPotterDBREND
plot_evolution_NN_entity('DBRSTARTdbr:JKRowlingDBREND', pcas, models, colors, subdirectory = 'Papetti')

# Frequency Harry Potter e rowling
print("frequenza parole Harry Potter e rowling")
entities = ['DBRSTARTdbr:JKRowling', 'DBRSTARTdbr:HarryPotterDBREND']
printOccurancies(entities)


# Similarità harry e rowling
print("Similarità harry e rowling: ")
word = 'DBRSTARTdbr:HarryPotterDBREND'
entities = ['DBRSTARTdbr:JKRowlingDBREND']
results = calculateSimilarityOverYears(word, entities)
print(results)
print()

# Più simili a Harry Potter
plot_evolution_NN_entity('DBRSTARTdbr:HarryPotterDBREND', pcas, models, colors, subdirectory = 'Papetti')



# Spostamento università
print("frequenza universities")
entities = ['DBRSTARTdbr:MassachusettsInstituteofTechnologyDBREND', 'DBRSTARTdbr:ColumbiaUniversityDBREND', 'DBRSTARTdbr:PrincetonUniversityDBREND', 'DBRSTARTdbr:NewYorkUniversityDBREND', 'DBRSTARTdbr:StanfordUniversityDBREND', 'DBRSTARTdbr:UniversityofCaliforniaBerkeleyDBREND', 'DBRSTARTdbr:UniversityofChicagoDBREND']
printOccurancies(entities)


entity_name = 'DBRSTARTdbr:MassachusettsInstituteofTechnologyDBREND'
n_similar = 5

models = {}
models['1995'] = model1995
models['2000'] = model2000
models['2005'] = model2005
models['2010'] = model2010

similarities = {}

similarities['1995'] = get_trend_single_slice(entity_name, n_similar, models, '1995')
similarities['2000'] = get_trend_single_slice(entity_name, n_similar, models, '2000')
similarities['2005'] = get_trend_single_slice(entity_name, n_similar, models, '2005')
similarities['2010'] = get_trend_single_slice(entity_name, n_similar, models, '2010')

plot_trend_subplots(similarities, entity_name, subdirectory = 'Papetti')



entities = ['DBRSTARTdbr:MassachusettsInstituteofTechnologyDBREND', 'DBRSTARTdbr:ColumbiaUniversityDBREND', 'DBRSTARTdbr:PrincetonUniversityDBREND', 'DBRSTARTdbr:NewYorkUniversityDBREND', 'DBRSTARTdbr:StanfordUniversityDBREND', 'DBRSTARTdbr:UniversityofCaliforniaBerkeleyDBREND', 'DBRSTARTdbr:UniversityofChicagoDBREND']
plotPCAovertime(entities, "university", colors, subdirectory = 'Papetti')



pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
plot_evolution_NN_entity('DBRSTARTdbr:MassachusettsInstituteofTechnologyDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:ColumbiaUniversityDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:PrincetonUniversityDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:NewYorkUniversityDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:StanfordUniversityDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:UniversityofCaliforniaBerkeleyDBREND', pcas, models, colors, subdirectory = 'Papetti')
plot_evolution_NN_entity('DBRSTARTdbr:UniversityofChicagoDBREND', pcas, models, colors, subdirectory = 'Papetti')



# Vedere AOL come si evolve Evolution
print("frequenza AOL")
entities = ['DBRSTARTdbr:AOLDBREND', 'DBRSTARTdbr:GoogleDBREND']
printOccurancies(entities)
plot_evolution_NN_entity('DBRSTARTdbr:AOLDBREND', pcas, models, colors, subdirectory = 'Papetti')



# Insieme di nazioni vedere come si evolvono overtime
print("frequenza Nazioni") #"DBRSTARTdbr:NetherlandsDBREND", "DBRSTARTdbr:PolandDBREND",
entities = ["DBRSTARTdbr:UnitedKingdomDBREND", "DBRSTARTdbr:AustraliaDBREND", "DBRSTARTdbr:IranDBREND", "DBRSTARTdbr:IraqDBREND", "DBRSTARTdbr:UnitedStatesDBREND", "DBRSTARTdbr:ItalyDBREND", "DBRSTARTdbr:GermanyDBREND", "DBRSTARTdbr:FranceDBREND"]
printOccurancies(entities)
plotPCAovertime(entities, "States", colors, subdirectory = 'Papetti')


entities = ["DBRSTARTdbr:UnitedKingdomDBREND", "DBRSTARTdbr:AustraliaDBREND", "DBRSTARTdbr:IranDBREND", "DBRSTARTdbr:IraqDBREND", "DBRSTARTdbr:UnitedStatesDBREND"]
printOccurancies(entities)
plotPCAovertime(entities, "States2", colors, subdirectory = 'Papetti')


# **LTN**

In [0]:
### COSTANTI
# introduciamo le costanti (TUTTE, anche quelle che non useremo per trainare le LTN le entità) che saranno rappresentate nello spazio
# sintassi : ltnw.constant(nome, vettore_di_embedding)


### PREDICATI
# introduciamo qui i predicati(le reti neurali da addestrare)
# sintassi : ltnw.predicate(nome, num_input * dim_embedding)


### VARIABILI
# disponiamo delle variabili utili campionando lo spazio. Possiamo definire il campionamento a mano o passare un dominio
# sintassi :
#ltnw.variable("?a", np.random.uniform(min_dom, max_dom, (numero_campionamenti_spazio, embedding_dim)).astype("float32"));


### ASSIOMI
# introduciamo cose che conosciamo
# sintassi : 
#ltnw.axiom(predicato(costante)) 
#ltnw.axiom(~predicato(costante)) 
#ltnw.axiom(predicato1(costante1) | predicato2(costante2))


### QUANTIFICATORI
# quantificano le variabili (forall, exists)
# sintassi : ltnw.axiom("forall ?a: Predicate(?a) -> ~Predicate(?a)")


### INIZIALIZZAZIONE
#ltnw.initialize_knowledgebase(optimizer=tf.train.AdamOptimizer())
#ltnw.train(max_epochs = 1000, track_sat_levels=100, sat_level_epsilon=.99)

### QUERY
# sintassi : 
#ltnw.ask(predicato(costante))
#ltnw.ask(exists variabile: predicato(variabile))

In [0]:
# import models
model1995 = Word2Vec.load("model/Merged_1995.model")
model2000 = Word2Vec.load("model/Merged_2000.model")
model2005 = Word2Vec.load("model/Merged_2005.model")
model2010 = Word2Vec.load("model/Merged_2010.model")

file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_1995", 'rb')
data_1995 = pickle.load(file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2000", 'rb')
data_2000 = pickle.load(file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2005", 'rb')
data_2005 = pickle.load(file)
file.close()
file = open("/content/gdrive/My Drive/ProgettoAI/Merged_articles/data_2010", 'rb')
data_2010 = pickle.load(file)
file.close()

In [0]:
# need to ignore irrelevant entities
def extract_relevant_entities(data, embedding):
  relevant_entities = list()
  entities, _ = find_entities(data)
  for entity in entities:
    if entity in embedding:
      relevant_entities.append(entity)
  return relevant_entities

In [0]:
rel_ent_1995 = extract_relevant_entities(data_1995, model1995)
rel_ent_2000 = extract_relevant_entities(data_2000, model2000)
rel_ent_2005 = extract_relevant_entities(data_2005, model2005)
rel_ent_2010 = extract_relevant_entities(data_2010, model2010)

In [0]:
# set domain
domain1995 = np.array(list(map(lambda x : x, model1995[rel_ent_1995])))
domain2000 = np.array(list(map(lambda x : x, model2000[rel_ent_2000])))
domain2005 = np.array(list(map(lambda x : x, model2005[rel_ent_2005])))
domain2010 = np.array(list(map(lambda x : x, model2010[rel_ent_2010])))
domain = np.concatenate((domain1995, domain2000, domain2005, domain2010))

max_dom = np.amax(domain.flatten())
min_dom = np.amin(domain.flatten())

In [0]:
# clean entities names
def clean_entity_name(value):
  value = value.replace(":", "")
  # remove prefix and suffix
  value = value[11:]
  value = value[:-6]
  return value

## First batch 

In [0]:
ltnw._reset()
ltn_entities = set()

# DEFINE COSTANTS
# Need to load ALL entites that we will use, not only the ones uset for training the LTN
for entity in rel_ent_1995:
    ltnw.constant("{}1995".format(clean_entity_name(entity)), model1995[entity])
    ltn_entities.add("{}1995".format(clean_entity_name(entity)))
for entity in rel_ent_2000:
    ltnw.constant("{}2000".format(clean_entity_name(entity)), model2000[entity])
    ltn_entities.add("{}2000".format(clean_entity_name(entity)))
for entity in rel_ent_2005:
    ltnw.constant("{}2005".format(clean_entity_name(entity)), model2005[entity])
    ltn_entities.add("{}2005".format(clean_entity_name(entity)))
for entity in rel_ent_2010:
    ltnw.constant("{}2010".format(clean_entity_name(entity)), model2010[entity])
    ltn_entities.add("{}2010".format(clean_entity_name(entity)))

In [0]:
# DEFINE PREDICATE
ltnw.predicate("State", 1 * embedding_size)
ltnw.predicate("Company", 1 * embedding_size)
ltnw.predicate("Actor", 1 * embedding_size)
ltnw.predicate("Human", 1 * embedding_size)
ltnw.predicate("BornIn", 2 * embedding_size)

In [0]:
# DEFINE VARIABLES
# 250 is a good compromise between time and performance
ltnw.variable("?a", np.random.uniform(min_dom, max_dom, (250, embedding_size)).astype("float32"))
ltnw.variable("?b", np.random.uniform(min_dom, max_dom, (250, embedding_size)).astype("float32"))

In [0]:
# DEFINE AXIOMS
states = ["UnitedKingdom", "Netherlands", "Poland", "Australia", "Iran", "Iraq", "UnitedStates", "Italy", "Germany", "France"]
companies = ["AppleInc", "IBM", "Hewlett-Packard", "Dell", "Microsoft", "Amazoncom", "Intel", "AdvancedMicroDevices"]
actors = ["MarlonBrando", "JackNicholson", "RobertDeNiro", "AlPacino", "DanielDay-Lewis", "DustinHoffman", "TomHanks", "AnthonyHopkins"]
for state in states:
  for year in ["1995", "2000", "2005"]:
    if state + year in ltn_entities:
      ltnw.axiom("State({})".format(state + year))
      ltnw.axiom("~ Company({})".format(state + year))
      ltnw.axiom("~ Actor({})".format(state + year))

for company in companies:
  for year in ["1995", "2000", "2005"]:
    if company + year in ltn_entities:
      ltnw.axiom("Company({})".format(company + year))
      ltnw.axiom("~ State({})".format(company + year))
      ltnw.axiom("~ Actor({})".format(company + year))

for actor in actors:
  for year in ["1995", "2000", "2005"]:
    if actor + year in ltn_entities:
      ltnw.axiom("~ Company({})".format(actor + year))
      ltnw.axiom("~ State({})".format(actor + year))
      ltnw.axiom("Actor({})".format(actor + year))

ltnw.axiom("BornIn(MarlonBrando1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(MarlonBrando1995, Australia1995)")
ltnw.axiom(" ~ BornIn(MarlonBrando1995, Iran1995)")
ltnw.axiom("BornIn(JackNicholson1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(JackNicholson1995, France1995)")
ltnw.axiom(" ~ BornIn(JackNicholson1995, Iraq1995)")
ltnw.axiom("BornIn(RobertDeNiro1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(RobertDeNiro1995, SaudiArabia1995)")
ltnw.axiom(" ~ BornIn(RobertDeNiro1995, France1995)")
ltnw.axiom("BornIn(AlPacino1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(AlPacino1995, Netherlands1995)")
ltnw.axiom(" ~ BornIn(AlPacino1995, Poland1995)")
ltnw.axiom("BornIn(DanielDay-Lewis1995, UnitedKingdom1995)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis1995, SaudiArabia1995)")
ltnw.axiom("BornIn(DustinHoffman1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(DustinHoffman1995, France1995)")
ltnw.axiom(" ~ BornIn(DustinHoffman1995, Iran1995)")
ltnw.axiom("BornIn(TomHanks1995, UnitedStates1995)")
ltnw.axiom(" ~ BornIn(TomHanks1995, Italy1995)")
ltnw.axiom(" ~ BornIn(TomHanks1995, Germany1995)")
ltnw.axiom("BornIn(AnthonyHopkins1995, UnitedKingdom1995)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins1995, Italy1995)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins1995, Germany1995)")
ltnw.axiom("BornIn(MarlonBrando2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(MarlonBrando2000, Australia2000)")
ltnw.axiom(" ~ BornIn(MarlonBrando2000, Iran2000)")
ltnw.axiom("BornIn(JackNicholson2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(JackNicholson2000, France2000)")
ltnw.axiom(" ~ BornIn(JackNicholson2000, Iraq2000)")
ltnw.axiom("BornIn(RobertDeNiro2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(RobertDeNiro2000, SaudiArabia2000)")
ltnw.axiom(" ~ BornIn(RobertDeNiro2000, France2000)")
ltnw.axiom("BornIn(AlPacino2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(AlPacino2000, Netherlands2000)")
ltnw.axiom(" ~ BornIn(AlPacino2000, Poland2000)")
ltnw.axiom("BornIn(DanielDay-Lewis2000, UnitedKingdom2000)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis2000, SaudiArabia2000)")
ltnw.axiom("BornIn(DustinHoffman2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(DustinHoffman2000, France2000)")
ltnw.axiom(" ~ BornIn(DustinHoffman2000, Iran2000)")
ltnw.axiom("BornIn(TomHanks2000, UnitedStates2000)")
ltnw.axiom(" ~ BornIn(TomHanks2000, Italy2000)")
ltnw.axiom(" ~ BornIn(TomHanks2000, Germany2000)")
ltnw.axiom("BornIn(AnthonyHopkins2000, UnitedKingdom2000)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins2000, Italy2000)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins2000, Germany2000)")
ltnw.axiom("BornIn(MarlonBrando2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(MarlonBrando2005, Australia2005)")
ltnw.axiom(" ~ BornIn(MarlonBrando2005, Iran2005)")
ltnw.axiom("BornIn(JackNicholson2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(JackNicholson2005, France2005)")
ltnw.axiom(" ~ BornIn(JackNicholson2005, Iraq2005)")
ltnw.axiom("BornIn(RobertDeNiro2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(RobertDeNiro2005, SaudiArabia2005)")
ltnw.axiom(" ~ BornIn(RobertDeNiro2005, France2005)")
ltnw.axiom("BornIn(AlPacino2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(AlPacino2005, Netherlands2005)")
ltnw.axiom(" ~ BornIn(AlPacino2005, Poland2005)")
ltnw.axiom("BornIn(DanielDay-Lewis2005, UnitedKingdom2005)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(DanielDay-Lewis2005, SaudiArabia2005)")
ltnw.axiom("BornIn(DustinHoffman2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(DustinHoffman2005, France2005)")
ltnw.axiom(" ~ BornIn(DustinHoffman2005, Iran2005)")
ltnw.axiom("BornIn(TomHanks2005, UnitedStates2005)")
ltnw.axiom(" ~ BornIn(TomHanks2005, Italy2005)")
ltnw.axiom(" ~ BornIn(TomHanks2005, Germany2005)")
ltnw.axiom("BornIn(AnthonyHopkins2005, UnitedKingdom2005)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins2005, Italy2005)")
ltnw.axiom(" ~ BornIn(AnthonyHopkins2005, Germany2005)")

ltnw.axiom("forall ?a : State(?a) -> ~ Actor(?a) & ~ Company(?a)")
ltnw.axiom("forall ?a : Actor(?a) -> ~ State(?a) & ~ Company(?a)")
ltnw.axiom("forall ?a : Company(?a) -> ~ State(?a) & ~ Actor(?a)")
ltnw.axiom("forall ?a : Actor(?a) -> Human(?a)")
ltnw.axiom("forall ?a, ?b : BornIn(?a, ?b) -> Actor(?a) & State(?b)")

ltnw.axiom("forall ?a, ?b : BornIn(?a, ?b) -> ~ BornIn(?b, ?a)")

In [0]:
logger = logging.getLogger()
logger.basicConfig = logging.basicConfig(level=logging.DEBUG)

ltnw.initialize_knowledgebase(optimizer=tf.train.AdamOptimizer())
ltnw.train(max_epochs = 5000, track_sat_levels = 100)

In [0]:
unseen_states = ["Afghanistan", "Belgium", "Brazil", "Egypt", "Portugal"]
unseen_actors = ["DenzelWashington", "SeanPenn", "MorganFreeman", "JeffBridges"]
unseen_companies = ["Samsung", "Facebook", "Oracle", "Sony", "Motorola"]


with open("LTN.txt", "a") as out_file:
  print("Check if seen state in seen time slice is still a state")
  out_file.write("Check if seen state in seen time slice is still a state\n")
  for state in states:
    if "{}2000".format(state) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2000)".format(state))[0], 2))
      print("{} {}".format(state, pred))
      out_file.write("{} {} \n".format(state, pred))

  print("Check if seen state in unseen time slice is still a state")
  out_file.write("Check if seen state in unseen time slice is still a state\n")
  for state in states:
    if "{}2010".format(state) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2010)".format(state))[0], 2))
      print("{} {}".format(state, pred))
      out_file.write("{} {} \n".format(state, pred))

  print("Check if unseen state in seen time slice is still a state")
  out_file.write("Check if unseen state in seen time slice is still a state\n")
  for state in unseen_states:
    if "{}2000".format(state) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2000)".format(state))[0], 2))
      print("{} {}".format(state, pred))
      out_file.write("{} {} \n".format(state, pred))

  print("Check if unseen state in unseen time slice is still a state")
  out_file.write("Check if unseen state in unseen time slice is still a state\n")
  for state in unseen_states:
    if "{}2010".format(state) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2010)".format(state))[0], 2))
      print("{} {}".format(state, pred))
      out_file.write("{} {} \n".format(state, pred))




  print("Check if seen actor in seen time slice is still a actor")
  out_file.write("Check if seen actor in seen time slice is still a actor\n")
  for actor in actors:
    if "{}2000".format(actor) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2000)".format(actor))[0], 2))
      print("{} {}".format(actor, pred))
      out_file.write("{} {} \n".format(actor, pred))

  print("Check if seen actor in unseen time slice is still a actor")
  out_file.write("Check if seen actor in unseen time slice is still a actor\n")
  for actor in actors:
    if "{}2010".format(actor) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2010)".format(actor))[0], 2))
      print("{} {}".format(actor, pred))
      out_file.write("{} {} \n".format(actor, pred))

  print("Check if unseen actor in seen time slice is still a actor")
  out_file.write("Check if unseen actor in seen time slice is still a actor\n")
  for actor in unseen_actors:
    if "{}2000".format(actor) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2000)".format(actor))[0], 2))
      print("{} {}".format(actor, pred))
      out_file.write("{} {} \n".format(actor, pred))

  print("Check if unseen actor in unseen time slice is still a actor")
  out_file.write("Check if unseen actor in unseen time slice is still a actor\n")
  for actor in unseen_actors:
    if "{}2010".format(actor) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2010)".format(actor))[0], 2))
      print("{} {}".format(actor, pred))
      out_file.write("{} {} \n".format(actor, pred))



  print("Check if seen company in seen time slice is still a company")
  out_file.write("Check if seen company in seen time slice is still a company\n")
  for company in companies:
    if "{}2000".format(company) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2000)".format(company))[0], 2))
      print("{} {}".format(company, pred))
      out_file.write("{} {} \n".format(company, pred))

  print("Check if seen company in unseen time slice is still a company")
  out_file.write("Check if seen company in unseen time slice is still a company\n")
  for company in companies:
    if "{}2010".format(company) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2010)".format(company))[0], 2))
      print("{} {}".format(company, pred))
      out_file.write("{} {} \n".format(company, pred))

  print("Check if unseen company in seen time slice is still a company")
  out_file.write("Check if unseen company in seen time slice is still a company\n")
  for company in unseen_companies:
    if "{}2000".format(company) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2000)".format(company))[0], 2))
      print("{} {}".format(company, pred))
      out_file.write("{} {} \n".format(company, pred))

  print("Check if unseen company in unseen time slice is still a company")
  out_file.write("Check if unseen company in unseen time slice is still a company\n")
  for company in unseen_companies:
    if "{}2010".format(company) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2010)".format(company))[0], 2))
      print("{} {}".format(company, pred))
      out_file.write("{} {} \n".format(company, pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if types are separated correclty in seen time slice")
  out_file.write("Check if types are separated correclty in seen time slice\n")
  print("not Companies")
  out_file.write("not Companies\n")
  for element in states + actors + unseen_actors + unseen_states:
    if "{}2000".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2000)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))
  print("not States")
  out_file.write("not States\n")
  for element in companies + actors + unseen_companies + unseen_actors:
    if "{}2000".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2000)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))
  print("not Actors")
  out_file.write("not Actors\n")
  for element in states + companies + unseen_companies + unseen_states:
    if "{}2000".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2000)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if types are separated correclty in unseen time slice")
  out_file.write("Check if types are separated correclty in unseen time slice\n")
  print("not Companies")
  out_file.write("not Companies\n")
  for element in states + actors + unseen_actors + unseen_states:
    if "{}2010".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("Company({}2010)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))
  print("not States")
  out_file.write("not States\n")
  for element in companies + actors + unseen_companies + unseen_actors:
    if "{}2010".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("State({}2010)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))
  print("not Actors")
  out_file.write("not Actors\n")
  for element in states + companies + unseen_companies + unseen_states:
    if "{}2010".format(element) in ltn_entities:
      pred = str(round(ltnw.ask("Actor({}2010)".format(element))[0], 2))
      print("{} {}".format(element, pred))
      out_file.write("{} {} \n".format(element, pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if multy-entity relations are learned correctly")
  out_file.write("Check if multy-entity relations are learned correctly\n")
  pred = str(round(ltnw.ask("BornIn(MarlonBrando1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(MarlonBrando1995, Australia1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(MarlonBrando1995, Iran1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(JackNicholson1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(JackNicholson1995, France1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(JackNicholson1995, Iraq1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(RobertDeNiro1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(RobertDeNiro1995, SaudiArabia1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(RobertDeNiro1995, France1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(AlPacino1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AlPacino1995, Netherlands1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AlPacino1995, Poland1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(DanielDay-Lewis1995, UnitedKingdom1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DanielDay-Lewis1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DanielDay-Lewis1995, SaudiArabia1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(DustinHoffman1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DustinHoffman1995, France1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DustinHoffman1995, Iran1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(TomHanks1995, UnitedStates1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(TomHanks1995, Italy1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(TomHanks1995, Germany1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(AnthonyHopkins1995, UnitedKingdom1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AnthonyHopkins1995, Italy1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AnthonyHopkins1995, Germany1995)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if multy entity relations survives time")
  out_file.write("Check if multy entity relations survives time\n")
  pred = str(round(ltnw.ask("BornIn(MarlonBrando2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(MarlonBrando2010, Australia2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(MarlonBrando2010, Iran2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(JackNicholson2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(JackNicholson2010, France2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(JackNicholson2010, Iraq2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(RobertDeNiro2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(RobertDeNiro2010, SaudiArabia2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(RobertDeNiro2010, France2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(AlPacino2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AlPacino2010, Netherlands2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AlPacino2010, Poland2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(DanielDay-Lewis2010, UnitedKingdom2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DanielDay-Lewis2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DanielDay-Lewis2010, SaudiArabia2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(DustinHoffman2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DustinHoffman2010, France2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(DustinHoffman2010, Iran2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(TomHanks2010, UnitedStates2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(TomHanks2010, Italy2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(TomHanks2010, Germany2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask("BornIn(AnthonyHopkins2010, UnitedKingdom2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AnthonyHopkins2010, Italy2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))
  pred = str(round(ltnw.ask(" ~ BornIn(AnthonyHopkins2010, Germany2010)")[0], 2))
  print("{}".format(pred))
  out_file.write("{}\n".format(pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if multy entity relations works on unseen entities")
  out_file.write("Check if multy entity relations works on unseen entities\n")
  for actor in unseen_actors:
    pred = str(round(ltnw.ask("BornIn({}1995, UnitedStates1995)".format(actor))[0], 2))
    print("{} {}".format(actor, pred))
    out_file.write("{} {}\n".format(actor, pred))

  print("Check if multy entity relations survives time on unseen entities")
  out_file.write("Check if multy entity relations survives time on unseen entities\n")
  for actor in unseen_actors:
    pred = str(round(ltnw.ask("BornIn({}2010, UnitedStates2010)".format(actor))[0], 2))
    print("{} {}".format(actor, pred))
    out_file.write("{} {}\n".format(actor, pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if rules are learned")
  out_file.write("Check if rules are learned\n")
  rule = "forall ?a : State(?a) -> ~ Actor(?a) & ~ Company(?a)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))
  rule = "forall ?a : Actor(?a) -> ~ State(?a) & ~ Company(?a)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))
  rule = "forall ?a : Company(?a) -> ~ State(?a) & ~ Company(?a)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))
  rule = "forall ?a : Actor(?a) -> Human(?a)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))
  rule = "forall ?a, ?b : BornIn(?a, ?b) -> Actor(?a) & State(?b)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))
  rule = "forall ?a, ?b : BornIn(?a, ?b) -> ~ BornIn(?b, ?a)"
  pred = str(round(ltnw.ask(rule)[0], 2))
  print("{} {}".format(rule, pred))
  out_file.write("{} {}\n".format(rule, pred))

In [0]:
with open("LTN.txt", "a") as out_file:
  print("Check if seen unseen human is classified as actor")
  out_file.write("Check if seen unseen human is classified as actor\n")
  pred = str(round(ltnw.ask("Actor(BillClinton2000)")[0], 2))
  print("BillClinton {}".format(pred))
  out_file.write("BillClinton {} \n".format(pred))
  pred = str(round(ltnw.ask("Actor(BarackObama2000)")[0], 2))
  print("BarackObama {}".format(pred))
  out_file.write("BarackObama {} \n".format(pred))

## Second batch

In [0]:
ltnw._reset()
ltn_entities = set()

# DEFINE COSTANTS
# Need to load ALL entites that we will use, not only the ones uset for training the LTN
for entity in rel_ent_1995:
    ltnw.constant("{}1995".format(clean_entity_name(entity)), model1995[entity])
    ltn_entities.add("{}1995".format(clean_entity_name(entity)))
for entity in rel_ent_2000:
    ltnw.constant("{}2000".format(clean_entity_name(entity)), model2000[entity])
    ltn_entities.add("{}2000".format(clean_entity_name(entity)))
for entity in rel_ent_2005:
    ltnw.constant("{}2005".format(clean_entity_name(entity)), model2005[entity])
    ltn_entities.add("{}2005".format(clean_entity_name(entity)))
for entity in rel_ent_2010:
    ltnw.constant("{}2010".format(clean_entity_name(entity)), model2010[entity])
    ltn_entities.add("{}2010".format(clean_entity_name(entity)))

In [0]:
# DEFINE PREDICATE
ltnw.predicate("State", 1 * embedding_size)
ltnw.predicate("Human", 1 * embedding_size)
ltnw.predicate("NatoPresence", 1 * embedding_size)
ltnw.predicate("CivilWar", 1 * embedding_size)
ltnw.predicate("LeaderOf", 2 * embedding_size)

In [0]:
# DEFINE VARIABLES
# 250 is a good compromise between time and performance
ltnw.variable("?a", np.random.uniform(min_dom, max_dom, (250, embedding_size)).astype("float32"))
ltnw.variable("?b", np.random.uniform(min_dom, max_dom, (250, embedding_size)).astype("float32"))

In [0]:
states = ["BosniaandHerzegovina", "Albania", "Kosovo", "Afghanistan", 
          "Croatia", "Serbia", "UnitedStates", "Netherlands", "Poland", 
          "Australia", "Iran", "Iraq", "Italy", "Germany", 
          "France", "Slovenia", "Burundi"]
humans = ["Hamid_Karzai", "NicolasSarkozy", "JacquesChirac", "BarackObama",
          "HelmutKohl", "AngelaMerkel", "BillClinton", "GeorgeWBush"]

for state in states:
  for year in ["1995", "2000", "2005", "2010"]:
    if state + year in ltn_entities:
      ltnw.axiom("State({})".format(state + year))
      ltnw.axiom("~ Human({})".format(state + year))

for human in humans:
  for year in ["1995", "2000", "2005", "2010"]:
    if human + year in ltn_entities:
      ltnw.axiom("Human({})".format(human + year))
      ltnw.axiom("~ State({})".format(human + year))

ltnw.axiom("NatoPresence(Afghanistan2005)")
ltnw.axiom("NatoPresence(Afghanistan2010)")
ltnw.axiom("NatoPresence(Iraq2005)")
ltnw.axiom("NatoPresence(Iraq2010)")
ltnw.axiom("NatoPresence(BosniaandHerzegovina1995)")
ltnw.axiom("NatoPresence(BosniaandHerzegovina2000)")
ltnw.axiom("NatoPresence(Kosovo2000)")

ltnw.axiom("~ NatoPresence(Afghanistan2000)")
ltnw.axiom("~ NatoPresence(Iraq1995)")
ltnw.axiom("~ NatoPresence(BosniaandHerzegovina2005)")
ltnw.axiom("~ NatoPresence(Kosovo1995)")
ltnw.axiom("~ NatoPresence(France2000)")
ltnw.axiom("~ NatoPresence(Italy2005)")
ltnw.axiom("~ NatoPresence(UnitedStates1995)")
ltnw.axiom("~ NatoPresence(Germany2010)")


ltnw.axiom("CivilWar(Slovenia2000)")
ltnw.axiom("CivilWar(Croatia2000)")
ltnw.axiom("CivilWar(BosniaandHerzegovina2000)")
ltnw.axiom("CivilWar(Slovenia1995)")
ltnw.axiom("CivilWar(Croatia1995)")
ltnw.axiom("CivilWar(BosniaandHerzegovina1995)")
ltnw.axiom("CivilWar(Serbia1995)")
ltnw.axiom("CivilWar(Afghanistan2000)")
ltnw.axiom("CivilWar(Afghanistan1995)")
ltnw.axiom("CivilWar(Burundi1995)")
ltnw.axiom("CivilWar(Burundi2000)")
ltnw.axiom("CivilWar(Burundi2005)")
ltnw.axiom("CivilWar(Iraq1995)")

ltnw.axiom("~ CivilWar(France2000)")
ltnw.axiom("~ CivilWar(Italy2005)")
ltnw.axiom("~ CivilWar(UnitedStates1995)")
ltnw.axiom("~ CivilWar(Germany2010)")
ltnw.axiom("~ CivilWar(Australia2005)")
ltnw.axiom("~ CivilWar(UnitedKingdom1995)")
ltnw.axiom("~ CivilWar(UnitedStates2010)")
ltnw.axiom("~ CivilWar(Netherlands2010)")
ltnw.axiom("~ CivilWar(Iraq2010)")
ltnw.axiom("~ CivilWar(Afghanistan2010)")
ltnw.axiom("~ CivilWar(Serbia2005)")
ltnw.axiom("~ CivilWar(Slovenia2010)")


ltnw.axiom("LeaderOf(HamidKarzai2005, Afghanistan2005)")
ltnw.axiom("LeaderOf(HamidKarzai2010, Afghanistan2010)")
ltnw.axiom("LeaderOf(NicolasSarkozy2010, France2010)")
ltnw.axiom("LeaderOf(JacquesChirac1995, France1995)")
ltnw.axiom("LeaderOf(JacquesChirac2000, France2000)")
ltnw.axiom("LeaderOf(JacquesChirac2005, France2005)")
ltnw.axiom("LeaderOf(AngelaMerkel2005 , Germany2005)")
ltnw.axiom("LeaderOf(AngelaMerkel2010 , Germany2010)")
ltnw.axiom("LeaderOf(HelmutKohl1995 , Germany1995)")
ltnw.axiom("LeaderOf(BillClinton1995 , UnitedStates1995)")
ltnw.axiom("LeaderOf(BillClinton2000 , UnitedStates2000)")
ltnw.axiom("LeaderOf(GeorgeWBush2005 , UnitedStates2005)")
ltnw.axiom("LeaderOf(BarackObama2010 , UnitedStates2010)")


ltnw.axiom("~LeaderOf(JacquesChirac2000, Afghanistan2005)")
ltnw.axiom("~LeaderOf(BillClinton2000, Afghanistan2010)")
ltnw.axiom("~LeaderOf(AngelaMerkel2010, France2010)")
ltnw.axiom("~LeaderOf(BillClinton2000, France1995)")
ltnw.axiom("~LeaderOf(GeorgeWBush2005, France2000)")
ltnw.axiom("~LeaderOf(HelmutKohl1995, France2005)")
ltnw.axiom("~LeaderOf(JacquesChirac2000 , Germany2005)")
ltnw.axiom("~LeaderOf(AngelaMerkel2010 , Germany2010)")
ltnw.axiom("~LeaderOf(GeorgeWBush2005, Germany1995)")
ltnw.axiom("~LeaderOf(HamidKarzai2005 , UnitedStates1995)")
ltnw.axiom("~LeaderOf(JacquesChirac2000 , UnitedStates2000)")
ltnw.axiom("~LeaderOf(BillClinton2000 , UnitedStates2005)")
ltnw.axiom("~LeaderOf(AngelaMerkel2010 , UnitedStates2010)")


ltnw.axiom("forall ?a : State(?a) -> ~ Human(?a)")
ltnw.axiom("forall ?a : Human(?a) -> ~ State(?a) & ~ NatoPresence(?a)")
ltnw.axiom("forall ?a : NatoPresence(?a) -> State(?a)")
ltnw.axiom("forall ?a : CivilWar(?a) -> State(?a)")
ltnw.axiom("forall ?a, ?b : LeaderOf(?a, ?b) -> Human(?a) & State(?b)")
ltnw.axiom("forall ?a, ?b : LeaderOf(?a, ?b) -> ~ LeaderOf(?b, ?a)")

In [0]:
logger = logging.getLogger()
logger.basicConfig = logging.basicConfig(level=logging.DEBUG)

ltnw.initialize_knowledgebase(optimizer=tf.train.AdamOptimizer())
ltnw.train(max_epochs = 5000, track_sat_levels = 100)

In [0]:
with open("LTN2.txt", "a") as out_file:
  print("Check if the network has learned axioms")
  out_file.write("Check if the network has learned axioms\n")
  print("NatoPresence")
  out_file.write("NatoPresence\n")
  pred = str(round(ltnw.ask("NatoPresence(Afghanistan2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(Afghanistan2010)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(Iraq2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(Iraq2010)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(BosniaandHerzegovina1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(BosniaandHerzegovina2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("NatoPresence(Kosovo2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  print("\n")
  print("CivilWar")
  out_file.write("CivilWar\n")
  pred = str(round(ltnw.ask("CivilWar(Slovenia2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Croatia2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(BosniaandHerzegovina2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Slovenia1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Croatia1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(BosniaandHerzegovina1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Serbia1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Afghanistan2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Afghanistan1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Burundi1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Burundi2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Burundi2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("CivilWar(Iraq1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  print("\n")
  print("LeaderOf")
  out_file.write("LeaderOf\n")
  pred = str(round(ltnw.ask("LeaderOf(HamidKarzai2005, Afghanistan2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(HamidKarzai2010, Afghanistan2010)")[0],2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(NicolasSarkozy2010, France2010)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(JacquesChirac1995, France1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(JacquesChirac2000, France2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(JacquesChirac2005, France2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(AngelaMerkel2005 , Germany2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(AngelaMerkel2010 , Germany2010)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(HelmutKohl1995 , Germany1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(BillClinton1995 , UnitedStates1995)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(BillClinton2000 , UnitedStates2000)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(GeorgeWBush2005 , UnitedStates2005)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")
  pred = str(round(ltnw.ask("LeaderOf(BarackObama2010 , UnitedStates2010)")[0], 2))
  print(pred)
  out_file.write(pred + "\n")

Does not learn the relation LeaderOf

In [0]:
with open("LTN2.txt", "a") as out_file:
  print("Check if enities with semantic shifts invalidates NatoPresence predicates")
  out_file.write("Check if enities with semantic shifts invalidates NatoPresence predicates\n")
  ent = "Afghanistan1995"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Iraq1995"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "BosniaandHerzegovina2010"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Kosovo2005" # i was wrong, NATO is still in Kosovo, the net corrected me
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  print("Check if NatoPresence is not overfitting on all states")
  out_file.write("Check if NatoPresence is not overfitting on all states\n")
  ent = "France2005"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Italy2000"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "UnitedStates1995"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Germany2000"
  pred = str(round(ltnw.ask("NatoPresence({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))

In [0]:
with open("LTN2.txt", "a") as out_file:
  print("Check if enities with semantic shifts invalidates CivilWar predicates")
  out_file.write("Check if enities with semantic shifts invalidates CivilWar predicates\n")
  ent = "Slovenia2010"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Croatia2010"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "BosniaandHerzegovina2005" # state does not exist anymore, still make sense that it is represented near cilivwar articles
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Serbia2005"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Afghanistan2010"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Burundi2010" # always seen with civil war, not able to generalize
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Iraq2005"  # the net is wrong, no idea why
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))

In [0]:
with open("LTN2.txt", "a") as out_file:
  print("Check if unsees entity usually associated with CivilWar satisfies the predicate")
  out_file.write("Check if unsees entity usually associated with CivilWar satisfies the predicate\n")
  ent = "Rwanda1995"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Rwanda2000"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Rwanda2005"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Rwanda2010"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))

In [0]:
with open("LTN2.txt", "a") as out_file:
  print("Check if unsees entity without CivilWar war satisfies the predicate")
  out_file.write("Check if unsees entity without CivilWar war satisfies the predicate\n")
  ent = "Germany1995"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Germany2000"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Germany2005"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))
  ent = "Germany2010"
  pred = str(round(ltnw.ask("CivilWar({})".format(ent))[0], 2))
  print("{} {}".format(ent, pred))
  out_file.write("{} {}\n".format(ent, pred))

# **LTN RESULTS ANALYSIS**

In [0]:
# STATES analysis

print("frequenza Nazioni") 
entities = ["DBRSTARTdbr:UnitedKingdomDBREND", "DBRSTARTdbr:NetherlandsDBREND",
"DBRSTARTdbr:PolandDBREND", "DBRSTARTdbr:AustraliaDBREND", "DBRSTARTdbr:IranDBREND", "DBRSTARTdbr:IraqDBREND", "DBRSTARTdbr:UnitedStatesDBREND", "DBRSTARTdbr:ItalyDBREND", "DBRSTARTdbr:GermanyDBREND", "DBRSTARTdbr:FranceDBREND"]
printOccurancies(entities)

states = ['DBRSTARTdbr:PolandDBREND', 'DBRSTARTdbr:AustraliaDBREND', 'DBRSTARTdbr:UnitedStatesDBREND']
plotPCAovertime(states, "States", colors, subdirectory = 'LTN_Analysis')



pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
plot_evolution_NN_entity('DBRSTARTdbr:PolandDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')     #sono sempre vicini a stati
plot_evolution_NN_entity('DBRSTARTdbr:AustraliaDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:UnitedStatesDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')

print("frequenza Nazioni unseen") 
entities = ['DBRSTARTdbr:AfghanistanDBREND', 'DBRSTARTdbr:BelgiumDBREND', 'DBRSTARTdbr:BrazilDBREND','DBRSTARTdbr:EgyptDBREND', 'DBRSTARTdbr:PortugalDBREND']
printOccurancies(entities)

plotPCAovertime(entities, "States", colors, subdirectory = 'LTN_Analysis')

pcas = {1995: pca_1995, 2000: pca_2000, 2005: pca_2005, 2010: pca_2010}
models = {1995: model1995, 2000: model2000, 2005: model2005, 2010: model2010}
plot_evolution_NN_entity('DBRSTARTdbr:BelgiumDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')    
plot_evolution_NN_entity('DBRSTARTdbr:PortugalDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:AfghanistanDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:EgyptDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')


# ACTOR analysis

print("frequenza ATTORI") 
actors = ['DBRSTARTdbr:MarlonBrandoDBREND',
'DBRSTARTdbr:JackNicholsonDBREND', 
'DBRSTARTdbr:RobertDeNiroDBREND',
'DBRSTARTdbr:AlPacinoDBREND',
'DBRSTARTdbr:DanielDay-LewisDBREND',
'DBRSTARTdbr:DustinHoffmanDBREND',
'DBRSTARTdbr:TomHanksDBREND',
'DBRSTARTdbr:AnthonyHopkinsDBREND']

printOccurancies(actors)
plot_evolution_NN_entity('DBRSTARTdbr:DustinHoffmanDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:DanielDay-LewisDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:AnthonyHopkinsDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')

unseenActor = ['DBRSTARTdbr:DenzelWashingtonDBREND',
'DBRSTARTdbr:SeanPennDBREND',
'DBRSTARTdbr:MorganFreemanDBREND', 
'DBRSTARTdbr:JeffBridgesDBREND'] 

print("frequenza UNSEEN ATTORI") 
printOccurancies(unseenActor)



print("frequenza COMPANY") 
company = [
'DBRSTARTdbr:AppleIncDBREND' ,
'DBRSTARTdbr:IBMDBREND' ,
'DBRSTARTdbr:Hewlett-PackardDBREND' ,
'DBRSTARTdbr:DellDBREND' ,
'DBRSTARTdbr:MicrosoftDBREND' , 
'DBRSTARTdbr:AmazoncomDBREND' , 
'DBRSTARTdbr:IntelDBREND' ,
'DBRSTARTdbr:AdvancedMicroDevicesDBREND']

plot_evolution_NN_entity('DBRSTARTdbr:OracleDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')
plot_evolution_NN_entity('DBRSTARTdbr:DellDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')

printOccurancies(company)

print("frequenza UNSEEEN COMPANY") 
unseenCompany = [
'DBRSTARTdbr:SamsungDBREND' ,
'DBRSTARTdbr:OracleDBREND' ,
'DBRSTARTdbr:SonyDBREND' ,
'DBRSTARTdbr:MotorolaDBREND' ,
'DBRSTARTdbr:FacebookDBREND']

printOccurancies(unseenCompany)
plot_evolution_NN_entity('DBRSTARTdbr:MotorolaDBREND', pcas, models, colors, subdirectory = 'LTN_Analysis')